In [ ]:
import pandas as pd
# Проверим данные, полученные с сайта investing.com
df = pd.read_csv('Best Buy Stock Price History.csv',index_col=['Date'], parse_dates=['Date']).sort_index()
df

,Price,Open,High,Low,Vol.,Change %
Date,,,,,,
2016-01-04,30.24,29.44,30.66,29.36,6.34M,0.67%
2016-01-05,30.73,30.31,30.78,30.08,4.73M,1.62%
2016-01-06,29.55,30.12,30.32,29.34,8.34M,-3.84%
2016-01-07,29.82,29.16,30.26,28.98,5.96M,0.91%
2016-01-08,28.56,29.34,29.38,28.25,7.06M,-4.23%
...,...,...,...,...,...,...
2018-12-24,48.24,48.53,51.01,47.72,1.78M,-1.27%
2018-12-26,51.57,48.60,51.62,48.46,4.42M,6.90%
2018-12-27,51.96,50.72,51.98,50.00,3.49M,0.76%


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.ar_model import AutoReg
from statsmodels.tsa.stattools import adfuller

# Восстанавливаем частоту, если данные дневные
df = df.asfreq('B')  # 'B' означает "бизнес-дни" (исключает выходные)

# Заполняем пропуски, если они есть (например, методом вперед)
df['Price'] = df['Price'].ffill()

y = df['Price']

# Функция для проверки стационарности по обратным корням
def is_stationary(model):
    # Для AR модели корни характеристического уравнения:
    # 1 - phi1*z - phi2*z^2 - ... - phi_p*z^p = 0
    # Модель считается стационарной, если все обратные корни (model.roots)
    # имеют модуль больше 1.
    roots = model.roots
    return np.all(np.abs(roots) > 1), roots


table = 80 

# Перебираем модели от AR(1) до AR(7)
for p in range(1, 8):
    print(f"\n{'='*table}\n{f'Модель AR({p})':^{table}}\n{'='*table}")
    # Три варианта спецификации:
    # 'n'  - без константы и тренда
    # 'c'  - с константой без тренда
    # 'ct' - с константой и трендом
    for trend, label in zip(['n', 'c', 'ct'], 
                            ['без константы и тренда', 'с константой без тренда', 'с константой и трендом']):
        try:
            model = AutoReg(y, lags=p, trend=trend, old_names=False).fit()
            print(f"\n--- AR({p}) модель ({label}) ---")
            display(model.summary())
            
            # Проверка стационарности по обратным корням
            stationary, roots = is_stationary(model)
            print("Обратные корни модели:", roots)
            if stationary:
                print("Модель удовлетворяет условию стационарности (все корни вне единичного круга).")
            else:
                print("Модель НЕ удовлетворяет условию стационарности!")
            
            # Дополнительно: ADF-тест на остатках модели
            adf_result = adfuller(model.resid)
            print(f"ADF-тест (остатки): статистика = {adf_result[0]:.4f}, p-value = {adf_result[1]:.4f}")
            
        except Exception as e:
            print(f"Не удалось оценить модель AR({p}) с вариантом ({label}). Ошибка: {e}")


                                  Модель AR(1)                                  

--- AR(1) модель (без константы и тренда) ---


<class 'statsmodels.iolib.summary.Summary'>
"""
                            AutoReg Model Results                             
==============================================================================
Dep. Variable:                  Price   No. Observations:                  781
Model:                     AutoReg(1)   Log Likelihood               -1238.437
Method:               Conditional MLE   S.D. of innovations              1.184
Date:                Wed, 19 Feb 2025   AIC                           2480.873
Time:                        10:52:53   BIC                           2490.192
Sample:                    01-05-2016   HQIC                          2484.457
                         - 12-31-2018                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Price.L1       1.0002      0.001   1320.565      0.000       0.999       1.002
                                    Roots                                    
=============================================================================
                  Real          Imaginary           Modulus         Frequency
-----------------------------------------------------------------------------
AR.1            0.9998           +0.0000j            0.9998            0.0000
-----------------------------------------------------------------------------
"""

Обратные корни модели: [0.99983693]
Модель НЕ удовлетворяет условию стационарности!
ADF-тест (остатки): статистика = -15.3457, p-value = 0.0000

--- AR(1) модель (с константой без тренда) ---


<class 'statsmodels.iolib.summary.Summary'>
"""
                            AutoReg Model Results                             
==============================================================================
Dep. Variable:                  Price   No. Observations:                  781
Model:                     AutoReg(1)   Log Likelihood               -1237.091
Method:               Conditional MLE   S.D. of innovations              1.182
Date:                Wed, 19 Feb 2025   AIC                           2480.181
Time:                        10:52:53   BIC                           2494.159
Sample:                    01-05-2016   HQIC                          2485.557
                         - 12-31-2018                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.2367      0.144      1.642      0.101      -0.046       0.519
Price.L1       0.9961      0.003    386.792      0.000       0.991       1.001
                                    Roots                                    
=============================================================================
                  Real          Imaginary           Modulus         Frequency
-----------------------------------------------------------------------------
AR.1            1.0039           +0.0000j            1.0039            0.0000
-----------------------------------------------------------------------------
"""

Обратные корни модели: [1.00389498]
Модель удовлетворяет условию стационарности (все корни вне единичного круга).
ADF-тест (остатки): статистика = -15.3200, p-value = 0.0000

--- AR(1) модель (с константой и трендом) ---


...

Обратные корни модели: [-1.51900803-0.95272169j -1.51900803+0.95272169j -0.23347035-1.62069034j
 -0.23347035+1.62069034j  0.99980985-0.j          1.50628799-0.77904693j
  1.50628799+0.77904693j]
Модель НЕ удовлетворяет условию стационарности!
ADF-тест (остатки): статистика = -27.7778, p-value = 0.0000

--- AR(7) модель (с константой без тренда) ---


<class 'statsmodels.iolib.summary.Summary'>
"""
                            AutoReg Model Results                             
==============================================================================
Dep. Variable:                  Price   No. Observations:                  781
Model:                     AutoReg(7)   Log Likelihood               -1222.826
Method:               Conditional MLE   S.D. of innovations              1.175
Date:                Wed, 19 Feb 2025   AIC                           2463.652
Time:                        10:52:54   BIC                           2505.516
Sample:                    01-13-2016   HQIC                          2479.760
                         - 12-31-2018                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.2421      0.145      1.671      0.095      -0.042       0.526
Price.L1       0.9250      0.036     25.740      0.000       0.855       0.995
Price.L2       0.0472      0.049      0.964      0.335      -0.049       0.143
Price.L3       0.1173      0.049      2.394      0.017       0.021       0.213
Price.L4      -0.0779      0.049     -1.585      0.113      -0.174       0.018
Price.L5      -0.0345      0.049     -0.703      0.482      -0.131       0.062
Price.L6      -0.0207      0.049     -0.421      0.674      -0.117       0.076
Price.L7       0.0397      0.036      1.101      0.271      -0.031       0.110
                                    Roots                                    
=============================================================================
                  Real          Imaginary           Modulus         Frequency
-----------------------------------------------------------------------------
AR.1           -1.5209           -0.9552j            1.7959           -0.4107
AR.2           -1.5209           +0.9552j            1.7959            0.4107
AR.3           -0.2334           -1.6234j            1.6400           -0.2727
AR.4           -0.2334           +1.6234j            1.6400            0.2727
AR.5            1.0038           -0.0000j            1.0038           -0.0000
AR.6            1.5127           -0.7784j            1.7012           -0.0756
AR.7            1.5127           +0.7784j            1.7012            0.0756
-----------------------------------------------------------------------------
"""

Обратные корни модели: [-1.5208748 -0.95517421j -1.5208748 +0.95517421j -0.23341069-1.62335231j
 -0.23341069+1.62335231j  1.00377278-0.j          1.51266095-0.77838977j
  1.51266095+0.77838977j]
Модель удовлетворяет условию стационарности (все корни вне единичного круга).
ADF-тест (остатки): статистика = -27.7712, p-value = 0.0000

--- AR(7) модель (с константой и трендом) ---


<class 'statsmodels.iolib.summary.Summary'>
"""
                            AutoReg Model Results                             
==============================================================================
Dep. Variable:                  Price   No. Observations:                  781
Model:                     AutoReg(7)   Log Likelihood               -1222.821
Method:               Conditional MLE   S.D. of innovations              1.175
Date:                Wed, 19 Feb 2025   AIC                           2465.642
Time:                        10:52:54   BIC                           2512.158
Sample:                    01-13-2016   HQIC                          2483.539
                         - 12-31-2018                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.2586      0.220      1.174      0.240      -0.173       0.690
trend       5.726e-05      0.001      0.100      0.920      -0.001       0.001
Price.L1       0.9250      0.036     25.739      0.000       0.855       0.995
Price.L2       0.0472      0.049      0.963      0.335      -0.049       0.143
Price.L3       0.1173      0.049      2.395      0.017       0.021       0.213
Price.L4      -0.0777      0.049     -1.581      0.114      -0.174       0.019
Price.L5      -0.0346      0.049     -0.703      0.482      -0.131       0.062
Price.L6      -0.0207      0.049     -0.422      0.673      -0.117       0.076
Price.L7       0.0389      0.037      1.056      0.291      -0.033       0.111
                                    Roots                                    
=============================================================================
                  Real          Imaginary           Modulus         Frequency
-----------------------------------------------------------------------------
AR.1           -1.5244           -0.9587j            1.8008           -0.4107
AR.2           -1.5244           +0.9587j            1.8008            0.4107
AR.3           -0.2336           -1.6272j            1.6439           -0.2727
AR.4           -0.2336           +1.6272j            1.6439            0.2727
AR.5            1.0045           -0.0000j            1.0045           -0.0000
AR.6            1.5222           -0.7766j            1.7088           -0.0751
AR.7            1.5222           +0.7766j            1.7088            0.0751
-----------------------------------------------------------------------------
"""

Обратные корни модели: [-1.52443674-0.95868344j -1.52443674+0.95868344j -0.23364461-1.62716355j
 -0.23364461+1.62716355j  1.00449848-0.j          1.52217671-0.77661207j
  1.52217671+0.77661207j]
Модель удовлетворяет условию стационарности (все корни вне единичного круга).
ADF-тест (остатки): статистика = -27.7705, p-value = 0.0000
